In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import WebDriverException
from tqdm.auto import tqdm

import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import time
import os

In [2]:
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
service = Service(executable_path=ChromeDriverManager().install())

In [3]:
def crawl_tistory_links(keyword_input):
''' input에 검색키워드를 넣으면 tistory에서 검색하여
    url 링크들을 따오는 함수
'''
    browser = webdriver.Chrome(service=service, options=chrome_options)
    browser.maximize_window()
    url = f"https://www.tistory.com/m/search?query={keyword_input}&tab=post"
    browser.get(url)
    browser.implicitly_wait(2)
    
    for i in range(20): #스크롤 20번 내림 약 560개의 url 가져옴
        browser.find_element(By.CSS_SELECTOR, "body").send_keys(Keys.END)
        time.sleep(1)
        browser.execute_script("return window.scrollY")
    
    links = browser.find_elements(By.CSS_SELECTOR, "ul.sch_list_post > li > a")
    links = [_.get_attribute('href') for _ in links]
    
    browser.quit()
    
    return links

In [5]:
def crawl_tistory_blogs(links):
'''
crawl_tistory_links로 뽑아온 url을 담은 links를 이용
BS로 요청하여 title, text를 뽑아옴
주석처리는 selenium 사용했던 것들
'''
    crwl_blog_text = []
    titles = []
#    browser = webdriver.Chrome(service=service, options=chrome_options)
#    browser.set_page_load_timeout(7)
    # browser.Manage().Timeouts().PageLoad = TimeSpan.FromMinutes(3)
    tk0 = tqdm(range(len(links)), smoothing=0, mininterval=1.0)
    for i in tk0:
        try:
            # browser.set_page_load_timeout(10)
            blog_text = ''
            # browser.implicitly_wait(2.5)
            #browser.get(links[i])
            
            #WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
            response = requests.get(links[i])
            soup = BeautifulSoup(response.text, 'html.parser')
            #soup = BeautifulSoup(browser.page_source, 'html.parser')
            title = soup.find("h3", class_="tit_blogview")
            div = soup.find("div", class_="blogview_content useless_p_margin editor_ke") # 본문위 class 종류가 2개라 2개 다 불러오고 합침
            div2 = soup.find("div", class_="blogview_content editor_ke")#
            combined_str = str(div) + str(div2)
            combined_soup = BeautifulSoup(combined_str, 'html.parser')
            text = combined_soup.find_all('p')
            text = [_ for _ in text if not _.find('a')]
            for t in text:
                blog_text += t.text
            if blog_text is not None: #본문 text가 존재하면 append 아니면 'None'을 추가
                crwl_blog_text.append(blog_text)
            else:
                crwl_blog_text.append('None')
            if title is not None: #title가 존재하면 append 아니면 'None'을 추가
                titles.append(title.text)
            else:
                titles.append('None')
        except WebDriverException:
            print(links[i],'WebDriverException에러 발생')
            pass
        except TimeoutException:
            print('Timeout에러 발생')
            pass
        except requests.exceptions.ConnectionError:
            print(links[i],"connectionError 발생")
            pass
    crwl_blog_text = [re.sub('\n+', '',text) for text in crwl_blog_text] #전처리
    
    return crwl_blog_text, titles

In [7]:
search_keywords = ['ENFJ 특징'] #검색키워드
for keyword in search_keywords:
    links = crawl_tistory_links(keyword)
    crwl_blog_text, titles = crawl_tistory_blogs(links)
    df = pd.DataFrame({'title': titles, 'content': crwl_blog_text})
    df.to_csv(f'{keyword[:4]}_tistory_crawling.csv',index=False)

  0%|          | 0/535 [00:00<?, ?it/s]

https://11231004.com/m/entry/enfj-%EC%9C%A0%ED%98%95-%ED%8A%B9%EC%A7%95-%EC%9E%A5%EB%8B%A8%EC%A0%90-%ED%8C%8C%EC%95%85%ED%95%98%EA%B8%B0 connectionError 발생
https://mbti.icrdt.com/m/49 connectionError 발생
https://mbti.icrdt.com/m/26 connectionError 발생
https://11231004.com/m/entry/ISFJ-%ED%8A%B9%EC%A7%95-%EA%B6%81%ED%95%A9-%EC%9E%90%EC%84%B8%ED%95%98%EA%B2%8C-%EC%95%8C%EC%95%84%EB%B3%B4%EA%B8%B0 connectionError 발생


In [109]:
df = pd.read_csv('ISFP_tistory_crawling.csv')
df.content

0     오늘은 MBTI 유형중, ISFP 유형의 특징을 총정리하는 시간을 갖도록 하겠습니다...
1      엠비티아이 잇프피 유형의 특징에 대해 알아보도록 하겠습니다. ISFP 유형은 조용...
2       ISFP는 강한 예술적 측면을 가지고 있습니다. 그들은 시각 예술, 음악 또는 ...
3     MBTI 유형 중에서도 조용하고 차분한 성격인 ISFP유형입니다. 이 유형은 내향형...
4     MBTI 성격 유형 중 ISFP는 "호기심 많은 예술가"로 알려져 있습니다. 이들은...
5      목차 ISFP유형은 Myers-Briggs 유형 지표(MBTI)에서 사용되는 성격...
6     ISFP 유형 특징에 대해서 알아보도록 할게요!ISFP-ISFP 특징--ISFP 장...
7     대체적으로 보이는 특징은 아래와 같아요.●감성적: ISFP는 자신의 감정을 중요하게...
8     호기심 많은 예술가, 성인군자형인 ISFP입니다.말없이 다정하고 온화하며 사람들에게...
9     ISFP 특징 팩폭 연예인 등 ISFP에 대해 알아보도록 하겠습니다.MBTI는 My...
10                                                  NaN
11    isfp유형의문의 인싸라고 불리는 ISFP유형은 I(내향형) + S(감각형) + F...
12    ISFP 유형은 조용하고 친절하며인내심이 강한 성격입니다.말보다 행동으로 나타내는 ...
13    매우 섬세하며 따뜻한 배려가 정말 매력적인 여자들입니다. 친절하고 겸손하며 늘 상대...
14    ISFP는 민감하고, 창의적이며, 그들의 감정에 깊이 적응합니다. 관계에서 ISFP...
15    이번 게시글에서는 ISFP 특징에 대해 알아보도록 하겠다. ISFP는 내향적 감각형...
16    [ISFP 유형] 특징 총정리 ISFP형은 점잖고 감수성이 많으며 인정이 많습니다....
17    이번 게시글에서는 ISFP 남자 특징에 대해 알아보도록 하겠다. ISFP 남